In [423]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score



In [424]:
data = pd.read_csv("airline-passengers.csv")
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


## Split Data

Training 0.8 dan Test 0.2

In [425]:
training_set = data.iloc[:115, 1:2].values
test_set = data.iloc[115:, 1:2].values
training_set =pd.DataFrame(training_set)
test_set =pd.DataFrame(test_set)

print((training_set).shape , (test_set).shape)

(115, 1) (29, 1)


## PreProsessing Minmax

In [426]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
test_set_scaled = sc.fit_transform(test_set)
type(training_set_scaled)


numpy.ndarray

## transform to a supervised learning problem


In [427]:
# # Creating a data structure with 60 time-steps and 1 output
# X_train = []
# y_train = []
# x = 4
# for i in range(x, 115):
#     X_train.append(training_set_scaled[i-x:i, 0])
#     y_train.append(training_set_scaled[i, 0])
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# print(X_train)
# # print(y_train)

In [428]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

# print(df.shape)
# transform to a supervised learning problem
X_train, y_train = split_sequence(training_set_scaled, 3) 
X_test, y_test = split_sequence(test_set_scaled, 3) 


In [429]:
newX_train = pd.DataFrame(X_train.reshape(112,3),columns=['Xt-2','Xt-1','Xt'])
newY_train = pd.DataFrame(y_train, columns=['Output'])
newX_test = pd.DataFrame(X_test.reshape(26,3),columns=['Xt-2','Xt-1','Xt'])
newY_test = pd.DataFrame(y_test, columns=['Output'])

In [430]:
new_data_passenger_test = pd.concat([newX_test,newY_test],axis=1)
new_data_passenger_train = pd.concat([newX_train,newY_train],axis=1)
new_data_passenger_train  

,Xt-2,Xt-1,Xt,Output
0,0.020672,0.036176,0.072351,0.064599
1,0.036176,0.072351,0.064599,0.043928
2,0.072351,0.064599,0.043928,0.080103
3,0.064599,0.043928,0.080103,0.113695
4,0.043928,0.080103,0.113695,0.113695
...,...,...,...,...
107,0.599483,0.609819,0.552972,0.666667
108,0.609819,0.552972,0.666667,0.630491
109,0.552972,0.666667,0.630491,0.669251
110,0.666667,0.630491,0.669251,0.855297


## Seleksi Fitur

In [431]:
y_train =new_data_passenger_train['Output'] 
X_train = new_data_passenger_train.drop(columns=['Output'])
y_test =new_data_passenger_test['Output'] 
X_test = new_data_passenger_test.drop(columns=['Output'])

## Eksekusi Model KNN Regresi

In [444]:
# # pjng = 30
# # # for i in range(pjng)
# #     neigh = KNeighborsRegressor(n_neighbors=i)
# #     neigh.fit(X, y)
# #     print(neigh.predict(y))


neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)
y_pred =neigh.predict(X_test)
print(y_pred,array(y_test))

[0.22609819 0.09302326 0.0994832  0.14470284 0.1369509  0.28552972
 0.22739018 0.31395349 0.5        0.71317829 0.88888889 0.47674419
 0.43281654 0.25968992 0.2881137  0.34625323 0.29198966 0.38501292
 0.47286822 0.46770026 0.71317829 0.96899225 0.85658915 0.57364341
 0.51550388 0.48578811] [0.         0.08653846 0.16025641 0.1025641  0.30769231 0.27564103
 0.3525641  0.51923077 0.76282051 0.79807692 0.49038462 0.31089744
 0.16666667 0.30448718 0.34294872 0.25961538 0.34935897 0.48397436
 0.51923077 0.72115385 1.         0.94871795 0.63461538 0.48397436
 0.25641026 0.39102564]
